<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <div class="crop">
            <img src="http://networthcorp.net/NetworthCorp_LogoUsage_R1-01.jpg" style="height:75px; margin-left:0px" />
        </div>
    </div>
    <div style="width: 59%; display: inline-block">
            <h1  style="text-align: center">Blocking</h1><br>
        <div style="width: 90%; text-align: center; display: inline-block;"><i>Author:</i> <strong>NetworthCorp</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="Enter Date" pubdate>July, 2018</time>
        </div>
    </div>
</div>

## || Functionality:
__This Function cluster similar entities into blocks so that pair-wise comparisons are executed only between co-occurring entities.__

## || Input Parameters:
* __source:__ It takes the binned dataframe of a source as input.
* __row_pair_agg:__ It Determines how row pair aggregation is done. It can take:
   * __avg:__ Average of the all the columns.
   * __wtavg:__ Weighted Average of the columns.
* __cutoff:__ It takes a cutoff of similarity to reduce the number of blocks. ( Default: 80% )

## || Imports:

* __block_config_file:__ It stores the weights of column for row pair aggregation.<hr>
__Note:__ Edit this file for every source.

## || Return:
__This Function returns a Dataframe where every row stores the index of rows in Source file and represent a block.__

## || Code:

In [466]:
import numpy as np
import pandas as pd
import math
from block_config_file import column_weight as w

In [459]:
def blocking(source,row_pair_agg='avg',cutoff=80):
    df=source
    pair=pd.DataFrame(columns = ['i','j']) #Initialising 2 variables i and j for index matching between rows.
    n=len(df) #Initialising n to find number of rows in a dataframe.

    for column in df:
        for i in range(n):
            for j in range(i,n):

                if((df.iloc[i][column])==(df.iloc[j][column])):
                    pair=pair.append({'i': int(i),'j':int(j),column:1},ignore_index=True)
                else:
                    pair=pair.append({'i': int(i),'j':int(j),column:0},ignore_index=True)

    row_sim_score = pair.apply(lambda x: pd.Series(x.dropna().values))
    row_sim_score = row_sim_score.dropna()
    
    if row_pair_agg=='avg':
        
        row_sim_score['Percent Same'] = round(((row_sim_score.iloc[:, 2:].mean(axis=1))*100),0)
        sim_score1 = row_sim_score.pivot(index='j', columns='i', values='Percent Same')
        sim_score2 = row_sim_score.pivot(index='i', columns='j', values='Percent Same')
        sim_score1.values[[np.arange(n)]*2] = np.nan
        sim_score1[sim_score1.isnull()] = sim_score2
    
    elif row_pair_agg=='wtavg':
        
        wt=[]
        for column in df:
            if column in w:
                wt.append(w.get(column))
            else:
                wt.append(1)
        
        data= row_sim_score.iloc[:, 2:]
        wtavg=pd.DataFrame(columns=['Percent Same'])

        for index, row in data.iterrows():
            wtavg=wtavg.append({'Percent Same':round(((np.average(row, weights=wt))*100),0)},ignore_index=True)

        row_sim_score =pd.concat([row_sim_score, wtavg], axis=1)
        sim_score1 = row_sim_score.pivot(index='j', columns='i', values='Percent Same')
        sim_score2 = row_sim_score.pivot(index='i', columns='j', values='Percent Same')
        sim_score1.values[[np.arange(n)]*2] = np.nan
        sim_score1[sim_score1.isnull()] = sim_score2   
    
    db= sim_score1
    n=len(db)
    block=pd.DataFrame()
    for column in db:
        val=db[column].tolist()
        ind=int(column)
        del val[ind]
        maxm=max(val)
        y=db[column][db[column]>=maxm].index.tolist()
        x=list(y)
        x.remove(ind)
        for value in x:
            sim=db[column][value]
            sim_val=db[value].tolist()
            sim_ind=int(value)
            del sim_val[sim_ind]
            per=max(sim_val)
            if ((per>sim)==True):
                y.remove(value)
        #print(y)
        block = block.append(pd.Series(y), ignore_index=True)

    n=len(block)
    x=[]
    for i in range(n):
        for j in range(i+1,n):
            if (set(list(block.iloc[j,:].dropna()))<=set(list(block.iloc[i,:].dropna())))==True:
                x.append(j)
            elif (set(list(block.iloc[i,:].dropna()))<=set(list(block.iloc[j,:].dropna())))==True:
                x.append(i)

    x=list(set(x))
    block=block.drop(block.index[x])
    length=pd.DataFrame(columns=['index','length'])
    for index, row in block.iterrows():
        leng=len(list(row.dropna()))
        length=length.append({'index':index,'length':leng},ignore_index=True)

    length=length.sort_values('length')#,ascending=False)
    length.index = range(len(length.index))
    leng=length['index'].tolist()
    block=block.reindex(leng)
    block.index = range(len(block.index))

    for index, row in block.iterrows():
        value=list(row.dropna())
        if(len(value)==1):
            ind=int(value[0])
            val=db[ind].tolist()
            del val[ind]
            maxm=max(val)
            y=db[ind][db[ind]>=maxm].index.tolist()
            block.iloc[index,:]=np.nan
            block=block.append(pd.Series(y), ignore_index=True)
    block=block.dropna(how='all')
    block.index = range(len(block.index))

    block_len=True
    while (block_len==True):
        n=len(block)
        for i in range(n): 
            for j in range(i+1,n):
                set1=list(block.iloc[i,:].dropna())
                set2=list(block.iloc[j,:].dropna())
                inter=list(set(set1).intersection(set2))
                leng=len(inter)
                if leng>0:
                    y=list(set(set1+set2))
                    block.iloc[i,:]=np.nan
                    block.iloc[j,:]=np.nan
                    block=block.append(pd.Series(y), ignore_index=True)

        block=block.dropna(how='all')
        block.index = range(len(block.index))
        blk_len=len(block) 
        if(n==blk_len):
            block_len=False
        else:
            block_len=True

    block_len=True
    while (block_len==True):
        n=len(block)
        for i in range(n):
            value=list(block.iloc[i,:].dropna())
            if(len(value)>0):
                sim_match=[]
                for x in value:
                    y=db[x][db[x]>=cutoff].index.tolist()
                    sim_match.extend(y)
            sim_match=list(set(sim_match))

            for j in range(i+1,n):
                set1=list(block.iloc[j,:].dropna())
                set2=sim_match
                inter=list(set(set1).intersection(set2))
                leng=len(inter)
                if leng>0:
                    y=list(set(set1+set2))
                    block.iloc[i,:]=np.nan
                    block.iloc[j,:]=np.nan
                    block=block.append(pd.Series(y), ignore_index=True)

        block=block.dropna(how='all')
        block.index = range(len(block.index))
        blk_len=len(block) 
        if(n==blk_len):
            block_len=False
        else:
            block_len=True
            
    return(block)

In [462]:
#Binning:-
x=binning('pairwise.csv')

## || Test 1:

In [463]:
blocking(x,'wtavg')

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,15.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,26.0,28.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,56.0,41.0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6.0,11.0,52.0,54.0,55.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,34.0,35.0,23.0,27.0,30.0,31.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## || Test 2:

In [464]:
blocking(x,'avg')

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,43.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0,...,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0


## || Test 3:

In [465]:
blocking(x,'avg',60)

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0
